# Image Captioning - ResNet-LSTM model

*Last Updated: 01/31/2022*

This note book use a ResNet-LSTM model for image captioning


**References**
 
1. Udacity's Computer Vision Nanodegree
2. https://github.com/cocodataset/cocoapi
3. https://colab.research.google.com/github/rammyram/image_captioning/blob/master/Image_Captioning.ipynb
4. https://arxiv.org/pdf/1411.4555.pdf

**Prerequisite**

Please connect to your Google drive before running this notebook.

Please download the COCO dataset with the following notebook:

https://github.com/sliao-mi-luku/Image-Captioning/blob/main/ImageCaptioning_download_COCO2014.ipynb

## Import Libraries

In [ ]:
import numpy as np
import torch
import torch.utils.data

import torch.nn as nn

from torchvision import transforms
from torchvision import models

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, BatchSampler

from torchtext.data.metrics import bleu_score

from PIL import Image

import nltk
nltk.download('punkt')

import sys
sys.path.append(""/opt/cocoapi/PythonAPI')

import math
import time
import pickle
import json
import os
import urllib
import zipfile
from tqdm import tqdm
from copy import deepcopy

from pycocotools.coco import COCO

import random

from collections import Counter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
cuda


## Vocabulary Object

Vocabular object adapted from Udacity Computer Vision Nanodegree

In [ ]:
class Vocabulary(object):
    """
    Vocabulary object (Implemented by Udacity)
    """
    def __init__(self, vocab_threshold, vocab_file='/content/vocab.pkl', start_word="<start>", end_word="<end>", unk_word="<unk>",
                 annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/captions_train2014.json',
                 vocab_from_file=False):
        """
        Args
            vocab_threshold: minimum count of the words to be considered a unique token
            vocab_file: vocab file
            start_word: start-of-sentence token
            end_word: end-of-sentence token
            unk_word: unknown-word token
            annotations_file: annotations file (for training dataset)
            vocab_from_file: (boolean) whether or not to use the existing vocab file
        """
        self.vocab_threshold = vocab_threshold
        self.vocab_file = vocab_file
        self.start_word = start_word
        self.end_word = end_word
        self.unk_word = unk_word
        self.annotations_file = annotations_file
        self.vocab_from_file = vocab_from_file
        self.get_vocab()

    def get_vocab(self):
        """
        Load/Create the vocab file
        """
        # load and use the existing vocab file
        if os.path.exists(self.vocab_file) & self.vocab_from_file:
            with open(self.vocab_file, 'rb') as f:
                vocab = pickle.load(f)
                self.word2idx = vocab.word2idx
                self.idx2word = vocab.idx2word
            print('Vocabulary successfully loaded from vocab.pkl file!')
        
        # build a new vocab file
        else:
            self.build_vocab()
            with open(self.vocab_file, 'wb') as f:
                pickle.dump(self, f)
    
    def build_vocab(self):
        """
        Create dicts for converting tokens to integers (and vice-versa)
        """
        self.init_vocab()
        self.add_word(self.start_word)
        self.add_word(self.end_word)
        self.add_word(self.unk_word)
        self.add_captions()

    def init_vocab(self):
        """
        Initialize the dictionaries for converting tokens to integers (and vice-versa)
        """
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        """
        Add a token to the vocabulary
        """
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def add_captions(self):
        """
        Loop over training captions and add all tokens to the vocabulary that meet or exceed the threshold
        """
        coco = COCO(self.annotations_file)
        counter = Counter()
        ids = coco.anns.keys()
        for i, id in enumerate(ids):
            caption = str(coco.anns[id]['caption'])
            tokens = nltk.tokenize.word_tokenize(caption.lower())
            counter.update(tokens)

            if i % 100000 == 0:
                print("[%d/%d] Tokenizing captions..." % (i, len(ids)))

        words = [word for word, cnt in counter.items() if cnt >= self.vocab_threshold]

        for i, word in enumerate(words):
            self.add_word(word)

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx[self.unk_word]
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

## Build Vocabulary

In [ ]:
# minimum count required to add to the vocabulary list
VOCAB_THRESHOLD = 5

In [ ]:
# build vocab file from training data
train_vocab = Vocabulary(vocab_threshold=VOCAB_THRESHOLD,
                         vocab_file='/content/vocab.pkl',
                         start_word="<start>",
                         end_word="<end>",
                         unk_word="<unk>",
                         annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/captions_train2014.json',
                         vocab_from_file=False)

loading annotations into memory...
Done (t=3.90s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...


## Data Transforms

In [ ]:
# training data transform
transform_train = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# validation/test data transform
transform_eval = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

## Custom COCO Datasets

There are 2 types of cumstom COCO dataset.

**Dev Mode**

When we loop over the `CoCoDataset_DevMode` dataset, the **processed image tensor** and the **tokenized captions** will be extracted

**Caption Mode**

When we loop over the `CoCoDataset_CaptionMode` dataset, the **original image matrix** (ndarray) and the **processed image tensor** will be extracted

### COCO Dataset (Dev Mode)

In [ ]:
class CoCoDataset_DevMode(torch.utils.data.Dataset):
    def __init__(self, transform, batch_size, vocab_file, annotations_file, img_folder):
        """
        Args
            transform: data transform
            batch_size: batch size
            vocab_file: path to the existing vocab file
            annotations_file: annotations file (for training dataset)
            img_folder: path to the images
        """
        # data transform
        self.transform = transform
        # batch size
        self.batch_size = batch_size
        # vocab
        self.vocab = vocab_file
        # image folder
        self.img_folder = img_folder
        # initialize COCO
        self.coco = COCO(annotations_file)
        # annotation ids
        self.ids = list(self.coco.anns.keys())
        print('Obtaining caption lengths...')
        all_tokens = [nltk.tokenize.word_tokenize(str(self.coco.anns[self.ids[index]]['caption']).lower()) for index in tqdm(np.arange(len(self.ids)))]
        self.caption_lengths = [len(token) for token in all_tokens]
        
    def __getitem__(self, idx):
        """
        Get the idx-th item from the dataset
        Returns
            image: processed image
            caption: tokenized caption (including special tokens)
        """
        ann_id = self.ids[idx]
        caption = self.coco.anns[ann_id]['caption']
        img_id = self.coco.anns[ann_id]['image_id']
        path = self.coco.loadImgs(img_id)[0]['file_name']
        # Convert image to tensor and pre-process using transform
        image = Image.open(os.path.join(self.img_folder, path)).convert('RGB')
        image = self.transform(image)
        # Convert caption to tensor of word ids.
        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(self.vocab(self.vocab.start_word))
        caption.extend([self.vocab(token) for token in tokens])
        caption.append(self.vocab(self.vocab.end_word))
        caption = torch.Tensor(caption).long()
        # return pre-processed image and caption tensors
        return image, caption

    def get_data_indices(self):
        # choose a length of the caption
        sel_length = np.random.choice(self.caption_lengths)
        # find all availalbe captions with this length
        all_indices = np.where([self.caption_lengths[i] == sel_length for i in np.arange(len(self.caption_lengths))])[0]
        # select batch_size captions among them
        indices = list(np.random.choice(all_indices, size=self.batch_size))
        return indices

    def __len__(self):
        return len(self.ids)

### Create Dev Mode Dataset for training and validation data

In [ ]:
BATCH_SIZE = 16

# training dataset for training purposes
dataset_train = CoCoDataset_DevMode(transform=transform_train,  # with augmentation
                                      batch_size=BATCH_SIZE,
                                      vocab_file=train_vocab,
                                      annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/captions_train2014.json',
                                      img_folder='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/images/train2014/')

training_batch_sampler = BatchSampler(sampler=SubsetRandomSampler(indices=dataset_train.get_data_indices()),
                                      batch_size=dataset_train.batch_size,
                                      drop_last=False)

dataloader_train = DataLoader(dataset=dataset_train, batch_sampler=training_batch_sampler)


# validation dataset for training purposes
dataset_val = CoCoDataset_DevMode(transform=transform_eval,  # no augmentation
                                  batch_size=BATCH_SIZE,
                                  vocab_file=train_vocab,
                                  annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/captions_val2014.json',
                                  img_folder='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/images/val2014/')

val_batch_sampler = BatchSampler(sampler=SubsetRandomSampler(indices=dataset_val.get_data_indices()),
                                 batch_size=dataset_val.batch_size,
                                 drop_last=False)

dataloader_val = DataLoader(dataset=dataset_val, batch_sampler=val_batch_sampler)


# training dataset for BLEU calculation
single_dataset_train = CoCoDataset_DevMode(transform=transform_eval,  # no augmentation
                                           batch_size=1,
                                           vocab_file=train_vocab,
                                           annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/captions_train2014.json',
                                           img_folder='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/images/train2014/')

single_dataloader_train = DataLoader(dataset=single_dataset_train, batch_size=1, shuffle=False)


# for BLEU calculation
single_dataset_val = CoCoDataset_DevMode(transform=transform_eval,  # no augmentation
                                         batch_size=1,
                                         vocab_file=train_vocab,
                                         annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/captions_val2014.json',
                                         img_folder='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/images/val2014/')

single_dataloader_val = DataLoader(dataset=single_dataset_val, batch_size=1, shuffle=False)

loading annotations into memory...
Done (t=0.79s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:42<00:00, 9835.79it/s] 


loading annotations into memory...
Done (t=1.68s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 202654/202654 [00:20<00:00, 9740.52it/s]


loading annotations into memory...
Done (t=0.67s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:42<00:00, 9798.51it/s]


loading annotations into memory...
Done (t=0.33s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 202654/202654 [00:20<00:00, 9855.94it/s]


In [ ]:
# size of vocab
VOCAB_SIZE = len(dataloader_train.dataset.vocab)
print("VOCAB_SIZE: {}".format(VOCAB_SIZE))

VOCAB_SIZE: 8855


### COCO Dataset (Caption Mode)

In [ ]:
class CoCoDataset_CaptionMode(torch.utils.data.Dataset):
    def __init__(self, transform, batch_size, vocab_file, annotations_file, img_folder):
        """
        Args
            transform: data transform
            batch_size: batch size
            vocab_file: path to the existing vocab file
            annotations_file: annotations file (for training dataset)
            img_folder: path to the images
        """
        # data transform
        self.transform = transform
         # batch size
        self.batch_size = batch_size
        # vocab
        self.vocab = vocab_file
        # image folder
        self.img_folder = img_folder
        # path
        test_info = json.loads(open(annotations_file).read())
        self.paths = [item['file_name'] for item in test_info['images']]
        
    def __getitem__(self, idx):
        """
        Get the idx-th item from the dataset
        Returns
            orig_image: original image
            image: preprocessed image
        """
        path = self.paths[idx]
        # Convert image to tensor and pre-process using transform
        PIL_image = Image.open(os.path.join(self.img_folder, path)).convert('RGB')
        orig_image = np.array(PIL_image)
        image = self.transform(PIL_image)
        # return original image and pre-processed image tensor
        return orig_image, image

    def __len__(self):
        return len(self.paths)

### Create Caption Mode Dataset for test data

In [ ]:
# for captioning demo
single_dataset_test = CoCoDataset_CaptionMode(transform=transform_eval,
                                              batch_size=1,
                                              vocab_file=train_vocab,
                                              annotations_file='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/annotations/image_info_test2014.json',
                                              img_folder='/content/drive/MyDrive/ImageCaptioning/opt/cocoapi/images/test2014/')

dataloader_test = DataLoader(dataset=single_dataset_test, batch_size=1, shuffle=False)

## Model

### CNN Encoder

In [ ]:
class EncoderCNN(nn.Module):
    """
    Encoder (CNN-based architecture)
    """
    def __init__(self, embed_size):
        """
        Args
            embed_size: (int) dimension of extracted image semantics features
        """
        super(EncoderCNN, self).__init__()
        # load the pre-trained ResNet
        resnet = models.resnet50(pretrained=True)
        # freeze the weights
        for param in resnet.parameters():
            param.requires_grad_(False)
        # grab all CNN layers except the last one
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        # embedding layers
        self.embedding = nn.Linear(resnet.fc.in_features, embed_size)

    def forward(self, images):
        """
        Forward pass
        Args
            images: (tensor) processed image tensor. shape=(batch_size, 3, 224, 224)
        Returns
            feature: (tensor) extracted image semantic features. shape=(batch_size, self.embed_size)
        """
        # resnet stage
        features = self.resnet(images)
        # flatten to 1 dim
        features = features.view(features.size(0), -1)
        # embedding to final feature
        features = self.embedding(features)
        return features

In [ ]:
## Create encoder
EMBED_SIZE = 512

encoder = EncoderCNN(embed_size=EMBED_SIZE)
print(encoder)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

EncoderCNN(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

### RNN Decoder

In [ ]:
class DecoderRNN(nn.Module):
    """
    Decoder (RNN-based architecture)
    """
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        """
        Args
            embed_size: (int) dimension of extracted image semantics features
            hidden_size: (int) dimension of decoder hidden states
            vocab_size: (int) size of vocabulary
            num_layers: (int) number of decoder layers
        """
        super(DecoderRNN, self).__init__()
        # embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # LSTM layer(s)
        self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        # dense layer from hidden states to vocab dimension
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, features, captions):
        """
        Forward pass
        Args
            features: (tensor) encoder output. shape=(batch_size, embed_size)
            captions: (tensor) caption tokens (each element is an int). shape=(batch_size, seq_len)
        Returns
            fc_output: (tensor) final output. shape=(batch, vocab_size)
        """
        # batch size
        batch_size = features.shape[0]
        # embedding dimension
        embed_size = features.shape[1]
        # caption length
        seq_len = captions.shape[1]
        # remove the <end> token
        captions = captions[:, :-1]
        # pass the tokenized captions into the embedding layer
        embedded_captions = self.embedding(captions)  # (batch_size, seq_len-1, embed_size)
        # convert features as the very first tokens
        features = torch.unsqueeze(features, dim=1)  # (batch_size, 1, embed_size)
        # concatenate to obtain lstm_input
        lstm_input = torch.cat((features, embedded_captions), dim=1)  # (batch_size, seq_len, embed_size)
        # LSTM layer
        lstm_output, lstm_hidden = self.lstm(lstm_input)
        # dense layer
        fc_output = self.fc(lstm_output)
        return fc_output
        

    def sample(self, inputs, states=None, max_len=20):
        """
        Decode an image from the embedded feature tensor.
        Args
            inputs: (tensor) embedded image features. shape=(1, 1, embed_size)
            states: (tensor) hidden states of LSTM. shape=(1, hidden_size)
            max_len: (int) maximum length of predicted token list
        Returns
            tokens: (list) a list of tokens predicted by decoder
        """
        tokens = []
        x = inputs
        # output tokens one by one
        for _ in range(max_len):
            # lstm layer
            x, states = self.lstm(x, states)  # (batch_size=1, 1, hidden_size)
            # dense layer
            x = self.fc(x)  # (batch_size=1, 1, vocab_size)
            # token
            tok = torch.argmax(x, dim=-1)  # (batch_size=1, 1)
            # append to the output
            tokens.append(int(tok[0, 0]))
            # early stop (token == 1)
            if tok[0, 0] == 1:
                break
            # embedding
            x = self.embedding(tok)  # (batch_size, 1, embed_size)
        return tokens

In [ ]:
## Create decoder
HIDDEN_SIZE = 512

decoder = DecoderRNN(embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, vocab_size=VOCAB_SIZE)
print(decoder)

DecoderRNN(
  (embedding): Embedding(8855, 512)
  (lstm): LSTM(512, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=8855, bias=True)
)


## Check Dataset

In [ ]:
def check_OSError(enc, dataloader, device):
    # number of steps per epoch
    steps_per_epoch = math.ceil(len(dataloader.dataset.caption_lengths)/dataloader.batch_sampler.batch_size)

    # iterate epoch
    for epoch in range(num_epochs):

        enc.eval()

        for i in range(steps_per_epoch):

            if (i+1) % 1000 == 0:
            print('\r' + "i = {}/{}".format(i+1, steps_per_epoch), end='')
            sys.stdout.flush()

            # sample indices from dataloader
            sampled_indices = dataloader.dataset.get_data_indices()

            # load
            dataloader.batch_sampler.sampler = SubsetRandomSampler(indices=sampled_indices)

            # load inputs
            images_t, _ = next(iter(dataloader))

            images_t = images_t.to(device)

            # encode
            try:
                features_t = enc(images_t)
            except OSError:
                print("OSError")
                print(sampled_indices)
                return
        
    return None

## Train the model

In [ ]:
def train_model(model_name, enc, dec, num_epochs, criterion, optimizer, device, print_every_steps=100, early_stop_epochs=10):
    """
    Train the model
    Args
        model_name: (str) unique name of the model to save
        enc: Pytorch DL model for encoder
        dec: Pytorch DL model for decoder
        num_epochs: number of epochs to train
        criterion: the loss function object
        optimizer: the optimizer
        print_every_steps: print the information every X steps
        early_stop_epochs: early stopping if the model doesn't improve after X epochs
    """
    # number of steps per epoch
    train_steps_per_epoch = math.ceil(len(dataloader_train.dataset.caption_lengths)/dataloader_train.batch_sampler.batch_size)
    val_steps_per_epoch = math.ceil(len(dataloader_val.dataset.caption_lengths)/dataloader_val.batch_sampler.batch_size)

    # iterate epoch
    for epoch in range(num_epochs):

        # time of start of epoch
        epoch_start_time = time.time()

        """
        Training
        """
        print("Training")

        enc.train()
        dec.train()

        for step_i in range(train_steps_per_epoch):
            # sample training indices from dataloader_train
            training_indices = dataloader_train.dataset.get_data_indices()
            # batch sampler
            new_sampler = SubsetRandomSampler(indices=training_indices)
            # load
            dataloader_train.batch_sampler.sampler = new_sampler

            # load inputs
            images_t, captions_t = next(iter(dataloader_train))

            images_t = images_t.to(device)
            captions_t = captions_t.to(device)

            # zero grad
            optimizer.zero_grad()

            # encode
            features_t = enc(images_t)

            # decode
            outputs_t = dec(features_t, captions_t)

            # loss
            loss = criterion(outputs_t.view(-1, VOCAB_SIZE), captions_t.view(-1))

            loss.backward()
            optimizer.step()

            # training stats
            stats = "Epoch {}/{}, Step {}/{}, Train Loss: {:4f}, Perplexity: {:5.4f}".format(epoch, num_epochs, step_i, train_steps_per_epoch, loss.item(), np.exp(loss.item()))

            # same line print out
            print('\r' + stats, end="")
            sys.stdout.flush()

            # new line print out
            if (step_i+1) % print_every_steps == 0:
                print('\r' + stats)

        """
        Evaluation Phase
        """
        print("Evaluation")

        enc.eval()
        dec.eval()

        eval_total_loss = 0.0

        for step_i in range(val_steps_per_epoch):
            # sample indices
            val_indices = dataloader_val.dataset.get_data_indices()
            # subset sampler
            dataloader_val.batch_sampler.sampler = SubsetRandomSampler(indices=val_indices)
            # load inputs
            images_t, captions_t = next(iter(dataloader_val))
            images_t.to(device)
            captions_t.to(device)

            with torch.no_grad():
                # encode
                features_t = enc(images_t)
                # decode
                outputs_t = dec(features_t, captions_t)
                # loss
                loss = criterion(outputs_t.view(-1, VOCAB_SIZE), captions_t.view(-1))
                eval_total_loss += loss.item() * features_t.size(0)

            # evaluation stats
            stats = "Epoch {}/{}, Step {}/{}, Evaluation Loss: {:4f}, Perplexity: {:5.4f}".format(epoch, num_epochs, step_i, val_steps_per_epoch, loss.item(), np.exp(loss.item()))

            # same line print out
            print('\r' + stats, end="")
            sys.stdout.flush()

            # new line print out
            if (step_i+1) % print_every_steps == 0:
                print('\r' + stats)
        
        eval_mean_loss = eval_total_loss / len(dataloader_val)
        print("Epoch {}/{}, Mean Eval Loss: {:4f}, Perplexity: {:5.4f}".format(epoch, num_epochs, step_i, val_steps_per_epoch, eval_mean_loss, np.exp(eval_mean_loss)))

        epoch_end_time = time.time()
        epoch_time_used = epoch_end_time - epoch_start_time
        # convert epoch_time_used into mm:ss
        mm = epoch_time_used // 60
        ss = epoch_time_used % 60

        # save encoder
        torch.save(enc.state_dict(), os.path.join('/content/drive/MyDrive/ImageCaptioning/saved_models', "encoder_" + model_name + ".pth"))
        torch.save(dec.state_dict(), os.path.join('/content/drive/MyDrive/ImageCaptioning/saved_models', "decoder_" + model_name + ".pth"))

    return enc, dec

### Train the model

In [ ]:
# move model to gpu
encoder = encoder.to(device)
decoder = decoder.to(device)

# unique model name for saving the weights
model_name = "020222"

# loss function
criterion = nn.CrossEntropyLoss()

# trainable parameters
params = list(decoder.parameters()) + list(encoder.embedding.parameters())

# optimizer
optimizer = torch.optim.Adam(params, lr=0.001)

# train
encoder, decoder = train_model(model_name, encoder, decoder, num_epochs=1, criterion=criterion, optimizer=optimizer, device=device, print_every_steps=100, early_stop_epochs=10)

Training


OSError: ignored

## Load Pretrained Weights

## Evaluation - BLEU Score

In [ ]:
def get_word_list_and_sentence(token_list):
    """
    Given a list of token (ex. [1, 1024, 222, 2]):
        1. remove the <start> token
        2. remove the <end> token and all its following tokens
    And finally return a list of words (ex. ["Hello", "world"]) and the complete sentence (ex. "Hello world")
    Args
        token_list: (list) a list of token integers
    Returns
        word_list: (list) a list of words
        sentence: (str) a str of the words joined by spaces 
    """
    word_list = []

    for tok in token_list:
        # skip the <start> token
        if tok == 0:
            continue
       
        # break if it's an <end> token
        if tok == 1:
            break
        
        # look up the word
        word = train_vocab.idx2word[tok]
        word_list.append(word)
    
    sentence = " ".join(word_list)
    
    return word_list, sentence

In [ ]:
def eval_BLEU(enc, dec, dataloader):
    """
    Evaluate the model on a dataset and calculate the overall BLEU score
    Args
        enc: (Pytorch model) encoder
        dec: (Pytorch model) decoder
        dataloader: (Pytorch dataloader) single_dataloader_train OR single_dataloader_val
    Returns
        all_pred_token_lists: (list of lists) list of predicted token lists (ex. [[1, 2, 3], [1, 4, 4, 3], ...])
        all_real_token_lists: (list of lists) list of acutual token lists (ex. [[1, 2, 3], [1, 4, 4, 3], ...])
    """

    enc.eval()
    dec.eval()

    enc.to(device)
    dec.to(device)

    all_pred_token_lists = []
    all_real_token_lists = []

    all_pred_word_lists = []
    all_real_word_lists = []

    bleu_sum = 0.0

    # load processed image and caption from dataloader (batch_size is 1)
    for image_t, caption_t in dataloader:

        with torch.no_grad():
            # encode
            feature_t = enc(image_t)
            # decode to a list of tokens
            token_list = dec.sample(feature_t)
            # convert token list to word list
            word_list = get_word_list_and_sentence(token_list)[0]
            # convert captions to references
            ref_list = 
            
            all_real_token_lists.append(caption_t[0].to_list())

        for token_list in all_pred_token_lists:
            all_pred_word_lists.append()

        for token_list in all_real_token_lists:
            all_real_word_lists.append(get_word_list_and_sentence(token_list)[0])
    
        # calculate BLEU
        bleu = bleu_score()
    
    return all_pred_token_lists, all_real_token_lists


In [ ]:
# validation dataset
all_pred_token_lists, all_real_token_lists = eval_BLEU(encoder, decoder, single_dataloader_val)

all
word_list, sentence = 